# Load libraries, define constants, functions, and classes

* libraries

In [1]:
import os

import sys
sys.path.append("../../2_train_and_test_models")

import numpy as np
np.random.seed(42)

import pandas as pd

from params import ROOT, GENOMES, TFS, SPECIES, Params
from collections import defaultdict
from sklearn.metrics import average_precision_score, roc_auc_score, confusion_matrix, log_loss, f1_score, accuracy_score

* constants

In [2]:
# Shorthand names for all model types to include in plots
MODELS = [
    "BM",
    "GRL",
    "MORALE"
]

# Plot-acceptable names for model types
MODEL_NAMES = {
    "BM-mm10": "Mouse-trained",
    "BM-hg38": "Human-trained",
    "GRL-mm10": "Mouse-trained (+GRL)",
    "GRL-hg38": "Human-trained (+GRL)",
    "MORALE-mm10": "Mouse-trained (+MORALE)",
    "MORALE-hg38": "Human-trained (+MORALE)"
}

# For simplicity, we will only consider the following repeat types
REPEAT_TYPES = ["LINE", "SINE"]

# if (test_species == "mm10"):
#     REPEAT_TYPES = ["DNA", "LINE", "Low_complexity", "LTR", "Other", "Satellite", "Simple_repeat", "SINE", "Unknown"]
#     # Removed due to < 500 instances in the test set: ["RC"]
# elif (test_species == "hg38"):
#     REPEAT_TYPES = ["DNA", "LINE", "Low_complexity", "LTR", "Simple_repeat", "SINE", "Unknown"]
#     # Removed due to < 500 instances in the test set: ["RC", "Retroposon", "RNA", "rRNA", "Satellite", "scRNA", "srpRNA", "tRNA"]
# else:
#     raise ValueError("Invalid test species")

SPECIES1 = "hg38"
SPECIES2 = "mm10"

* Helper functions

In [3]:
def get_preds_file(model, tf, source_species, domain):
    preds_root = f"{ROOT}/output"
    os.makedirs(preds_root, exist_ok=True)
    return f"{preds_root}/{model}_tf-{tf}_trained-{source_species}_tested-{domain}.preds.npy"

def get_labels_file(model, tf, source_species, domain):
    preds_root = f"{ROOT}/output"
    os.makedirs(preds_root, exist_ok=True)
    return f"{preds_root}/{model}_tf-{tf}_trained-{source_species}_tested-{domain}.labels.npy"

def load_fivefold_data(average=False, verbose=False):
    preds_dict      = defaultdict(lambda : defaultdict(lambda : dict()))
    labels_dict     = defaultdict(lambda : defaultdict(lambda : dict()))
    bound_indices   = defaultdict(lambda : defaultdict(lambda : dict()))
    unbound_indices = defaultdict(lambda : defaultdict(lambda : dict()))

    # Loop over mouse-trained, human-trained models, and domain-adaptive models
    for model in MODELS:
        for tf in TFS:
            for source in SPECIES:
                for target in SPECIES:
                    if verbose:
                        print(f"\t({model} on {tf} when: {source}-trained, and {target}-tested)")
                        
                    preds_file  = get_preds_file(model=model, tf=tf, source_species=source, domain=target)
                    labels_file = get_labels_file(model=model, tf=tf, source_species=source, domain=target)
                
                    try:
                        # Load them
                        preds = np.load(preds_file)
                        labels = np.load(labels_file)

                        # Calculate if we need to truncate the labels
                        if preds.shape[0] != labels.shape[0]:
                            print("\nTruncating labels\n")
                            labels = labels[:preds.shape[0]]

                        assert preds.shape[0] == labels.shape[0]

                        if average:
                            # We take the average of the sigmoid values across the five-folds
                            # to determine the confusion matrix
                            preds_dict[f"{model}-{source}"][tf][target] = np.mean(preds, axis=1)
                        else:                        
                            # We save predictions from each of the five-folds per model, TF, source, and target
                            preds_dict[f"{model}-{source}"][tf][target] = np.load(preds_file)
                        
                        labels_dict[f"{model}-{source}"][tf][target] = np.load(labels_file)

                        # Store unbound and bound indices for all models, TFs, sources, and targets
                        bound_indices[f"{model}-{source}"][tf][target]      = np.where(labels == 1)[0]
                        unbound_indices[f"{model}-{source}"][tf][target]    = np.where(labels == 0)[0]
                        
                    except:
                        print("Could not load regular preds/labels files")

    return preds_dict, labels_dict, bound_indices, unbound_indices

def generate_confusion_matrix(preds_dict, labels_dict, percents=False, differential=False, performance=False):
    # This function generates the full confusion matrix over predicitions for all models
    # that we care about. Additionally, we include the RAW number of differential predictions
    # (only errors, so type 1 or 2).

    #cnf_matrix = dict()
    cnf_matrix = defaultdict(lambda : defaultdict(lambda : dict()))

    # now go through each model and tf and calculate confusion matrix
    for model in MODELS:
        adapted_model_name                  = f"{model}-{SPECIES1}"
        ground_truth_model_name             = f"{model}-{SPECIES2}"

        for tf in TFS:
            for the_bound in [adapted_model_name, ground_truth_model_name]:

                if "BM" not in model and the_bound == ground_truth_model_name:
                    continue
                else:
                    cnf_matrix[the_bound][tf] = {"TP": 0, "FP": 0, "TN": 0, "FN": 0}

                    # (0) First we just calculate the raw confusion matrix for the model
                    preds   = preds_dict[the_bound][tf][SPECIES2]
                    labels  = labels_dict[the_bound][tf][SPECIES2]

                    # We need to categorize the adapted predictions based on the labels
                    tp_preds = preds[(preds > 0.5) & (labels == 1)]
                    fp_preds = preds[(preds > 0.5) & (labels == 0)]
                    tn_preds = preds[(preds <= 0.5) & (labels == 0)]
                    fn_preds = preds[(preds <= 0.5) & (labels == 1)]
                    
                    # (1) Now we add the raw counts for each category to the confusion matrix
                    if percents:
                        cnf_matrix[the_bound][tf]["TP"] = round((tp_preds.shape[0] / len(labels)) * 100, 3)
                        cnf_matrix[the_bound][tf]["FP"] = round((fp_preds.shape[0] / len(labels)) * 100, 3)
                        cnf_matrix[the_bound][tf]["TN"] = round((tn_preds.shape[0] / len(labels)) * 100, 3)
                        cnf_matrix[the_bound][tf]["FN"] = round((fn_preds.shape[0] / len(labels)) * 100, 3)
                    else:
                        cnf_matrix[the_bound][tf]["TP"] = tp_preds.shape[0]
                        cnf_matrix[the_bound][tf]["FP"] = fp_preds.shape[0]
                        cnf_matrix[the_bound][tf]["TN"] = tn_preds.shape[0]
                        cnf_matrix[the_bound][tf]["FN"] = fn_preds.shape[0]

                    # (2) We are primarily interested in gauging the differential false positive 
                    # predictions between the target model to the source model
                    if differential:

                        # Adapted models overpredict as compared to the ground truth model
                        overpred        = set(np.nonzero(preds - preds_dict[ground_truth_model_name][tf][SPECIES2] > 0.5)[0])
                        #overpred        = set(np.nonzero(preds - preds_dict["BM-mm10"][tf][SPECIES2] > 0.5)[0])
                        overpred_sites  = np.array([False if i not in overpred else True for i, j in enumerate(labels)])

                        # Adapted models underpredict as compared to the ground truth model
                        underpred       = set(np.nonzero(preds_dict[ground_truth_model_name][tf][SPECIES2] - preds > 0.5)[0])
                        #underpred       = set(np.nonzero(preds_dict["BM-mm10"][tf][SPECIES2] - preds > 0.5)[0])
                        underpred_list  = np.array([False if i not in underpred else True for i, j in enumerate(labels)])

                        if percents:
                            cnf_matrix[the_bound][tf]['dTP'] = round((preds[(preds > 0.5) & (labels == 1) & (overpred_sites == True)].shape[0] / tp_preds.shape[0]) * 100, 3)
                            cnf_matrix[the_bound][tf]['dFP'] = round((preds[(preds > 0.5) & (labels == 0) & (overpred_sites == True)].shape[0] / fp_preds.shape[0]) * 100, 3)
                            cnf_matrix[the_bound][tf]['dTN'] = round((preds[(preds <= 0.5) & (labels == 0) & (underpred_list == True)].shape[0] / tn_preds.shape[0]) * 100, 3)
                            cnf_matrix[the_bound][tf]['dFN'] = round((preds[(preds <= 0.5) & (labels == 1) & (underpred_list == True)].shape[0] / fn_preds.shape[0]) * 100, 3)
                        else:
                            cnf_matrix[the_bound][tf]['dTP'] = preds[(preds > 0.5) & (labels == 1) & (overpred_sites == True)].shape[0] / tp_preds.shape[0]
                            cnf_matrix[the_bound][tf]['dFP'] = preds[(preds > 0.5) & (labels == 0) & (overpred_sites == True)].shape[0] / fp_preds.shape[0]
                            cnf_matrix[the_bound][tf]['dTN'] = preds[(preds <= 0.5) & (labels == 0) & (underpred_list == True)].shape[0] / tn_preds.shape[0]
                            cnf_matrix[the_bound][tf]['dFN'] = preds[(preds <= 0.5) & (labels == 1) & (underpred_list == True)].shape[0] / fn_preds.shape[0]

                    # (3) We either get the performance metrics or we don't.
                    if performance:
                        
                        # We cannot just grab the auPRC from the confusion matrix calcs, because
                        # we averaged the sigmoid values over the five-folds. We need to load in the 
                        # performance data and do it manually.
                        performance_df = pd.read_csv(f"{ROOT}/plots/Tables1-2/performance_data.csv", index_col=None)
                        performance_df = performance_df.iloc[:, 1:]
                        
                        # We need to only grab the columns we need
                        performance_model_name = MODEL_NAMES[the_bound]
                        auPRC = performance_df.loc[performance_df["Model"] == performance_model_name, :].loc[performance_df["Eval"] == SPECIES2, :].loc[performance_df["TF"] == tf, :].loc[:, "auPRC"]
                        auPRC = round(np.mean(auPRC), 3)
                        
                        cnf_matrix[the_bound][tf]['auPRC'] = auPRC

    return cnf_matrix

In [4]:
def get_test_bed_file(tf, species):
    # This function returns the path to a BED-format file
    # containing the chromosome names, starts, and ends for
    # all examples to test the model with.
    # Note this is specific to a TF (binding labels
    # are loaded in from this file)!
    return(f"{ROOT}/data/{species}/{tf}/chr2.bed")

In [5]:
def make_preds_and_labels_dfs(preds_dict, labels_dict, repeat_labels):
    preds_dfs = defaultdict(lambda : dict())

    for model in MODELS:
        adapted_model_name = f"{model}-{SPECIES1}"
        for tf in TFS:
            dict_to_make_into_df = {"labels" : labels_dict[adapted_model_name][tf][SPECIES2]}
            goal_len = labels_dict[adapted_model_name][tf][SPECIES2].shape[0]  # assuming labels are already truncated

            # Now we get the relevant predictions
            model_preds = preds_dict[adapted_model_name][tf][SPECIES2]
            assert model_preds.shape[0] == goal_len

            dict_to_make_into_df[adapted_model_name] = model_preds

            for repeat_type in REPEAT_TYPES:
                dict_to_make_into_df[repeat_type] = repeat_labels[repeat_type][:goal_len]

            preds_dfs[tf][adapted_model_name] = pd.DataFrame(dict_to_make_into_df)

    return preds_dfs

In [6]:
def get_rmsk_file(test_species):
    return ROOT + f"/data/{test_species}/rmsk.bed"

def read_and_filter_rmsk_file(repeat_name, test_species, is_subfam=False, test_chrom="chr2"):
    # This function reads in the RepeatMasker bed file,
    # filters for only rows listing annotations of one
    # repeat type, and then returns only the start and 
    # end coordinate for each annotation.
    
    # We're assuming all the test set examples are on
    # one chromosome, so we don't need the first column.
    
    # assuming bed format
    filename = get_rmsk_file(test_species=test_species)
    df = pd.read_csv(filename, sep = "\t", usecols = [5, 6, 7, 10, 11], header = None)

    if is_subfam:
        df = df[df[10] == repeat_name]
        df = df[df[5] == test_chrom]
    else:
        df = df[df[11] == repeat_name]
        df = df[df[5] == test_chrom]

    sorted_repeat_coords = sorted(list(zip(df[6], df[7])), key = lambda tup : tup[0])
    return np.array(sorted_repeat_coords)

def get_repeat_and_test_set_overlap(list_a, list_b):
    # This function is similar to bedtools intersect,
    # but returns a binary yes/no for overlap for each
    # window in list_a.
    
    # Assumes everything's on the same chromosome
    # Assumes inputs are lists of 2-ples: (start, stop) 
    
    # output is list with len == len(list_a)
    matches = []
    b_index = 0
    for a_item in list_a:
        a_start, a_end = a_item
        while True:
            if b_index >= len(list_b):
                matches.append(False)
                break
                
            b_start, b_end = list_b[b_index]
            # the -1 is because bed files are 1-indexed
            if b_start > a_end - 1:  
                matches.append(False)
                break
            elif b_end <= a_start:
                b_index += 1
            else:
                matches.append(True)
                break
    assert len(matches) == len(list_a)
    return np.array(matches)

def get_test_bed_coords(test_species):
    # This function loads in the bed file for the test set
    # and keeps only the start and end coords for each entry.
    # Here we assume the test set is 1 chromosome
    
    # later analysis will assume the coords are sorted,
    # as in `sort -k1,1 -k2,2n $bed_file`
    
    # TF doesn't matter here because we're not using labels
    test_bed = get_test_bed_file(tf=TFS[0], species=test_species)
    df = pd.read_csv(test_bed, sep = "\t", usecols = [1, 2], header = None)
    return df.to_numpy()

def get_all_repeat_labels_and_indices(test_species):
    all_windows_coords  = get_test_bed_coords(test_species=test_species)
    repeat_labels       = dict()
    repeat_indices      = dict()

    for repeat_type in REPEAT_TYPES:
        
        print(repeat_type)

        repeat_type_coords = read_and_filter_rmsk_file(repeat_name=repeat_type, test_species=test_species)
        # filtering for repeat types with at least 500 instances
        # in the test set, so we don't get incorrectly extreme results

        assert len(repeat_type_coords) > 500, (repeat_type, len(repeat_type_coords))

        repeat_labels[repeat_type] = get_repeat_and_test_set_overlap(all_windows_coords, repeat_type_coords)
        
        repeat_indices[repeat_type] = set(np.nonzero(repeat_labels[repeat_type])[0])
        
    return repeat_labels, repeat_indices

def fix_repeat_name(repeat_name):
    # If a repeat name has an underscore in it, this will
    # mess up the latex formatting, so we replace the
    # underscores with spaces and then capitalize the first
    # letter of each word in the repeat name
    if "_" in repeat_name:
        return " ".join(repeat_name.split("_")).title()
    return repeat_name

In [7]:
def generate_repeat_table(preds_dfs, repeat_indices):
    data_matrix = dict()
    for model in MODELS:
        adapted_model_name = f"{model}-{SPECIES1}"
        data_matrix[adapted_model_name] = {i:{} for i in TFS}
        for tf in TFS:

            # (0) Instantiate an entry for each repeat type
            data_matrix[adapted_model_name][tf] = {i:{} for i in REPEAT_TYPES}

            # (0.5) Add a row for the eventual average across all repeat types
            data_matrix[adapted_model_name][tf]["Average"] = {}

            for rep in REPEAT_TYPES:

                # (1) Get the indices of the repeat type in the test set
                sites_with_repeat   = set([i for i, j in enumerate(preds_dfs[tf][adapted_model_name][rep])]).intersection(repeat_indices[rep])
                probs_rep           = preds_dfs[tf][adapted_model_name].iloc[[i for i in sites_with_repeat], :]
                preds_rep           = [1 if i > 0.5 else 0 for i in probs_rep[adapted_model_name]]
                labels_rep          = preds_dfs[tf][adapted_model_name]['labels'].iloc[[i for i in sites_with_repeat]]

                # (2) Now we use scikit to calculate different metrics for the model
                #data_matrix[adapted_model_name][tf][rep]["AUC"] = roc_auc_score(y_true=labels_rep, y_score=preds_rep)
                data_matrix[adapted_model_name][tf][rep]["performance"] = average_precision_score(y_true=labels_rep, y_score=preds_rep)

            # (4) Now add the average for each metric across all repeat types
            data_matrix[adapted_model_name][tf]["Average"]["performance"] = np.mean([data_matrix[adapted_model_name][tf][rep]["performance"] for rep in REPEAT_TYPES])

    return data_matrix

def print_table(preview_table, model_1, model_2, model_3, header=None, row_order=None, caption=None):
    print(r'\begin{table*}[t]{')
    print(r'\centering')

    if caption is not None:
        print(r'\caption{' + caption + r'\label{Tab:01}}')

    print(r'\resizebox{\textwidth}{!}{')
    print(r'\setlength{\tabcolsep}{0.8em}')

    reps = 3
    print(r'\centering \begin{tabular}{@{}ccccccccc@{}}\toprule')

    if header is None:
        header = "TF &"
        for i in REPEAT_TYPES:
            if i != REPEAT_TYPES[-1]:
                header += f" {i} &"
            else:
                header += f" {i}"
        header += f" & Average"
    
    col_order = list(REPEAT_TYPES)  # Create a copy to avoid modifying the original list
    col_order.extend(["Average"])

    if row_order is None:
        row_order = TFS
    last_row = row_order[-1]

    print(header + r' \\\midrule')

    table_segment = r'\begin{tabular}[c]{>{\centering\arraybackslash}p{1cm}>{\centering\arraybackslash}p{1cm}>{\centering\arraybackslash}p{1cm}}GRL & MORALE & Source\end{tabular}'  # Right after 1cm}, |>
    line = r'& ' + ' & '.join([table_segment] * reps) + r' \\'  # Note the parentheses
    print(line)

    # Initialize a dictionary to store sums for each column and model
    col_sums = {col: {model_1: 0, model_2: 0, model_3: 0} for col in col_order}

    for row_key in row_order:
        # Get model information
        row_model1 = [preview_table[model_1][row_key][col] for col in col_order]
        row_model2 = [preview_table[model_2][row_key][col] for col in col_order]
        row_model3 = [preview_table[model_3][row_key][col] for col in col_order]

        # Accumulate sums for averaging
        for i, col in enumerate(col_order):
            col_sums[col][model_1] += row_model1[i]['performance']
            col_sums[col][model_2] += row_model2[i]['performance']
            col_sums[col][model_3] += row_model3[i]['performance']
        
        # Find best model for each column
        best_models = {}
        for col_idx, col in enumerate(col_order):
            values = [row_model1[col_idx]['performance'], row_model2[col_idx]['performance'], row_model3[col_idx]['performance']]
            best_idx = np.argmax(values)
            
            if best_idx == 0:
                best_models[col] = model_1
            elif best_idx == 1:
                best_models[col] = model_2
            else:
                best_models[col] = model_3

        # Format model information, bolding the best
        row_model1_as_str = [
            (r"\textbf{" + str(round(num['performance'], 3)) + r"}" if model_1 == best_models[col] else str(round(num['performance'], 3)))
            for num, col in zip(row_model1, col_order)
        ]
        row_model2_as_str = [
            (r"\textbf{" + str(round(num['performance'], 3)) + r"}" if model_2 == best_models[col] else str(round(num['performance'], 3)))
            for num, col in zip(row_model2, col_order)
        ]
        row_model3_as_str = [
            (r"\textbf{" + str(round(num['performance'], 3)) + r"}" if model_3 == best_models[col] else str(round(num['performance'], 3)))
            for num, col in zip(row_model3, col_order)
        ]

        # Combine model information
        combined_row = [
            {model_1: i[0], model_2: i[1], model_3: i[2]}
            for i in zip(row_model1_as_str, row_model2_as_str, row_model3_as_str)
        ]
        combine_row_as_str = [
            r"\begin{tabular}[c]{>{\raggedleft\arraybackslash}p{1cm}>{\raggedleft\arraybackslash}p{1cm}>{\raggedleft\arraybackslash}p{1cm}}"
            + f"{i[model_1]} & {i[model_2]} & {i[model_3]}"
            r"\end{tabular}"
            for i in combined_row
        ]
        tf_fancy_name = TFS[TFS.index(row_key)]
        print(tf_fancy_name + " & " + " & ".join(combine_row_as_str) + r' \\')

    # Calculate and print the average row
    avg_row = {}
    num_rows = len(row_order)
    for col in col_order:
        avg_row[col] = {
            model: round(col_sums[col][model] / num_rows, 3)
            for model in [model_1, model_2, model_3]
        }
    
    # Find the best performing model for each column in the average row
    best_avg_models = {}
    for col in col_order:
        values = [avg_row[col][model_1], avg_row[col][model_2], avg_row[col][model_3]]
        best_idx = np.argmax(values)
            
        if best_idx == 0:
            best_avg_models[col] = model_1
        elif best_idx == 1:
            best_avg_models[col] = model_2
        else:
            best_avg_models[col] = model_3

    # Format the average row as strings, bolding the best
    avg_row_as_str = []
    for col in col_order:
        avg_row_as_str.append(
            r"\begin{tabular}[c]{>{\raggedleft\arraybackslash}p{1cm}>{\raggedleft\arraybackslash}p{1cm}>{\raggedleft\arraybackslash}p{1cm}}"
            + (
                r"\textbf{"
                + str(avg_row[col][model_1])
                + r"}"
                if model_1 == best_avg_models[col]
                else str(avg_row[col][model_1])
            )
            + " & "
            + (
                r"\textbf{"
                + str(avg_row[col][model_2])
                + r"}"
                if model_2 == best_avg_models[col]
                else str(avg_row[col][model_2])
            )
            + " & "
            + (
                r"\textbf{"
                + str(avg_row[col][model_3])
                + r"}"
                if model_3 == best_avg_models[col]
                else str(avg_row[col][model_3])
            )
            + r"\end{tabular}"
        )
    
    print(r"\midrule")
    print(r"Average & " + " & ".join(avg_row_as_str) + r' \\\bottomrule')

    print(r'\end{tabular}}{}')
    print(r'\end{table*}')

# Create repeat performance table

In [8]:
preds_dict, labels_dict, bound_indices, unbound_indices = load_fivefold_data(average=True, verbose=False)
repeat_labels, repeat_indices                           = get_all_repeat_labels_and_indices(test_species=SPECIES2)
preds_dfs                                               = make_preds_and_labels_dfs(preds_dict, labels_dict, repeat_labels)
cnf_matrix                                              = generate_confusion_matrix(preds_dict, labels_dict, percents=True, differential=False, performance=True)
repeat_table                                            = generate_repeat_table(preds_dfs, repeat_indices)

LINE
SINE


In [9]:
print_table(repeat_table, model_1=f"GRL-{SPECIES1}", model_2=f"MORALE-{SPECIES1}", model_3=f"BM-{SPECIES1}")

\begin{table*}[t]{
\centering
\resizebox{\textwidth}{!}{
\setlength{\tabcolsep}{0.8em}
\centering \begin{tabular}{@{}ccccccccc@{}}\toprule
TF & LINE & SINE & Average \\\midrule
& \begin{tabular}[c]{>{\centering\arraybackslash}p{1cm}>{\centering\arraybackslash}p{1cm}>{\centering\arraybackslash}p{1cm}}GRL & MORALE & Source\end{tabular} & \begin{tabular}[c]{>{\centering\arraybackslash}p{1cm}>{\centering\arraybackslash}p{1cm}>{\centering\arraybackslash}p{1cm}}GRL & MORALE & Source\end{tabular} & \begin{tabular}[c]{>{\centering\arraybackslash}p{1cm}>{\centering\arraybackslash}p{1cm}>{\centering\arraybackslash}p{1cm}}GRL & MORALE & Source\end{tabular} \\
CTCF & \begin{tabular}[c]{>{\raggedleft\arraybackslash}p{1cm}>{\raggedleft\arraybackslash}p{1cm}>{\raggedleft\arraybackslash}p{1cm}}0.103 & \textbf{0.124} & 0.111\end{tabular} & \begin{tabular}[c]{>{\raggedleft\arraybackslash}p{1cm}>{\raggedleft\arraybackslash}p{1cm}>{\raggedleft\arraybackslash}p{1cm}}0.102 & \textbf{0.129} & 0.104\end{tabul

-----